# PROJECT - 2

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import dateutil.parser

### Getting  information from a movie page

In [2]:
url = 'https://www.boxofficemojo.com/title/tt2488496/?ref_=bo_cso_table_1'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
# soup.prettify()

#### Movie Title

In [3]:
soup.find('title').text
title_string = soup.find('title').text
title_string

'Star Wars: Episode VII - The Force Awakens - Box Office Mojo'

In [4]:
title = title_string[:-18]
title

'Star Wars: Episode VII - The Force Awakens'

#### Domestic Gross

In [5]:
soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')

[<span class="money">$936,662,225</span>,
 <span class="money">$1,131,791,908</span>,
 <span class="money">$2,068,454,133</span>]

In [6]:
dom_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[0].text
dom_gross

'$936,662,225'

#### International Gross

In [7]:
intl_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[1].text
intl_gross

'$1,131,791,908'

#### Worldwide Gross

In [8]:
world_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[2].text
world_gross

'$2,068,454,133'

#### Domestic Openig Gross

In [9]:
dom_opening_gross = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[0]
dom_opening_gross

<span class="money">$247,966,675</span>

#### Budget

In [10]:
prod_budget = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[1]
prod_budget

<span class="money">$245,000,000</span>

#### Domestic Distributor

In [11]:
soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')

[<span class="money">$247,966,675</span>,
 <span class="money">$245,000,000</span>]

In [12]:
distributor_reggex = re.compile('Distributor')
soup.find(text=distributor_reggex)

'Domestic Distributor'

In [13]:
distr_string = find(text=distributor_reggex).findNext('span').text

NameError: name 'find' is not defined

In [14]:
dist_string = soup.find(text=distributor_reggex).findNext('span').text[:18]
dom_dist = dist_string
dom_dist

'Walt Disney Studio'

### Get Movie (findNext) Values

In [15]:
def get_movie_value(soup, field_name):
    
    """
    Get a value fro the movie page and take the string attribute of a movie on the page and return
    the value of the next sibling object or None if nothing is found.
    
    """
    obj = soup.find(text=re.compile(field_name))
    if not obj:
        return None
    next_element = obj.findNext()
    
    if next_element:
        return next_element.text
    else:
        return None
    

#### Domestic Opening Gross

In [16]:
dom_opening_gross = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[0].text
dom_opening_gross

'$247,966,675'

In [17]:
production_budget = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[1].text
production_budget

'$245,000,000'

#### Runtime

In [18]:
runtime = get_movie_value(soup, 'Running')
print(runtime)

2 hr 18 min


#### Rating

In [19]:
rating = get_movie_value(soup, 'MPAA')
print(rating)

PG-13


#### Release Date

In [20]:
rel_date = get_movie_value(soup, 'Release Date').split('\n')[0]
print(rel_date)

December 16, 2015


In [21]:
date = dateutil.parser.parse(rel_date)
date.month

12

In [21]:
date = dateutil.parser.parse(rel_date)
date.year


2015

In [22]:
import datetime

a = '2010-01-31'

datee = datetime.datetime.strptime(a, "%Y-%m-%d")


datee.month
Out[9]: 1

datee.year
Out[10]: 2010

datee.day
Out[11]: 31

#### Genres

In [23]:
genres = get_movie_value(soup, 'Genres').split()
genres

['Action', 'Adventure', 'Sci-Fi']

#### Distributor

In [24]:
distributor = get_movie_value(soup, 'Distributor')[:35]
distributor

'Walt Disney Studios Motion Pictures'

### Function to convert money and time string into integer values

In [25]:
def money_to_int(moneystring):
    
    """
    replaces the $ sign and removing the comma separator from the money string
    returns the integer value
    """
    
    moneystring = moneystring.replace('$', '').replace(',', '')
    return int(moneystring)

def runtime_to_minutes(runtimestring):
    
    """
    takes runtimestring in _hr_min format and returns it's value in number of minutes
    """
    
    runtime = runtimestring.split()
    try:
        minutes = int(runtime[0]) * 60 + int(runtime[2])
        return minutes
    except:
        return None
    
def to_date(datestring):
    date = dateutil.parser.parse(datestring)
    return date    

In [26]:
raw_dom_gross = dom_gross
domestic_gross = money_to_int(raw_dom_gross)

raw_intl_gross = intl_gross
international_gross = money_to_int(raw_intl_gross)

raw_world_gross = world_gross
worldwide_gross = money_to_int(raw_world_gross)

raw_opening_gross = dom_opening_gross
domestic_opening_gross = money_to_int(raw_opening_gross)

raw_budget = production_budget
budget = money_to_int(raw_budget)

raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_date = to_date(raw_release_date)
release_date

datetime.datetime(2015, 12, 16, 0, 0)

In [27]:
raw_runtime = get_movie_value(soup, 'Running')
runtime_minutes = runtime_to_minutes(raw_runtime)
runtime_minutes

138

#### Release date

In [28]:
raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
release_month = to_date(raw_release_date).month
release_month

12

In [29]:
release_month = rel_date.split()[0]
release_month

'December'

#### Actors

In [30]:
url = 'https://www.boxofficemojo.com/title/tt2488496/credits/?ref_=bo_tt_tab'
response = requests.get(url)
actor_page = response.text
actor_soup = BeautifulSoup(actor_page, 'lxml')

In [31]:
casts = actor_soup.find('table', id = 'principalCast').find_all('a', attrs={'class':'a-link-normal'})
actors = []
for cast in casts:
    cast = cast.text.split('\n')[0]
    actors += [cast]
actors

['Daisy Ridley', 'John Boyega', 'Oscar Isaac', 'Domhnall Gleeson']

### Getting the domestic lifetime ranking table

In [32]:
url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?ref_=bo_cso_ac'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')
# print(soup.prettify())

In [33]:
# finding the table
table = soup.find('table')
# table

In [34]:
rows = [row for row in table.find_all('tr')]
rows[1].find_all('td')

[<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank">1</td>,
 <td class="a-text-left mojo-field-type-title"><a class="a-link-normal" href="/title/tt2488496/?ref_=bo_cso_table_1">Star Wars: Episode VII - The Force Awakens</a></td>,
 <td class="a-text-right mojo-field-type-money">$936,662,225</td>,
 <td class="a-text-left mojo-field-type-year"><a class="a-link-normal" href="/year/2015/?ref_=bo_cso_table_1">2015</a></td>]

In [35]:
rows[1].find_all('td')[1].find('a')['href']

'/title/tt2488496/?ref_=bo_cso_table_1'

#### Previewing Lifetime Movie Data

In [36]:
movies = {}

for row in rows[1:201]:
    items = row.find_all('td')
    link = items[1].find('a')
    title = link.text
    url = link['href']
    movies[title] = [url]+ [i.text for i in items]

movies

{'Star Wars: Episode VII - The Force Awakens': ['/title/tt2488496/?ref_=bo_cso_table_1',
  '1',
  'Star Wars: Episode VII - The Force Awakens',
  '$936,662,225',
  '2015'],
 'Avengers: Endgame': ['/title/tt4154796/?ref_=bo_cso_table_2',
  '2',
  'Avengers: Endgame',
  '$858,373,000',
  '2019'],
 'Avatar': ['/title/tt0499549/?ref_=bo_cso_table_3',
  '3',
  'Avatar',
  '$760,507,625',
  '2009'],
 'Black Panther': ['/title/tt1825683/?ref_=bo_cso_table_4',
  '4',
  'Black Panther',
  '$700,426,566',
  '2018'],
 'Avengers: Infinity War': ['/title/tt4154756/?ref_=bo_cso_table_5',
  '5',
  'Avengers: Infinity War',
  '$678,815,482',
  '2018'],
 'Titanic': ['/title/tt0120338/?ref_=bo_cso_table_6',
  '6',
  'Titanic',
  '$659,363,944',
  '1997'],
 'Jurassic World': ['/title/tt0369610/?ref_=bo_cso_table_7',
  '7',
  'Jurassic World',
  '$652,295,625',
  '2015'],
 'The Avengers': ['/title/tt0848228/?ref_=bo_cso_table_8',
  '8',
  'The Avengers',
  '$623,357,910',
  '2012'],
 'Star Wars: Episode V

In [37]:
lifetime_movies = pd.DataFrame(movies).T
lifetime_movies.columns = ['link_stub', 'rank', 'title', 'domestic_gross','release_year']
lifetime_movies

,link_stub,rank,title,domestic_gross,release_year
Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015
Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019
Avatar,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009
Black Panther,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018
Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018
...,...,...,...,...,...
Fast Five,/title/tt1596343/?ref_=bo_cso_table_196,196,Fast Five,"$209,837,675",2011
Mission: Impossible - Ghost Protocol,/title/tt1229238/?ref_=bo_cso_table_197,197,Mission: Impossible - Ghost Protocol,"$209,397,903",2011
Wedding Crashers,/title/tt0396269/?ref_=bo_cso_table_198,198,Wedding Crashers,"$209,255,921",2005
Sherlock Holmes,/title/tt0988045/?ref_=bo_cso_table_199,199,Sherlock Holmes,"$209,028,679",2009


#### Storing movie page data into a dictionary

In [38]:
def get_movie_dict(link):
    """
    From BoxOfficeMojo link stub, request movie html, parse with BeautifulSoup, and
    collect 
        - title 
        - domestic gross
        - runtime 
        - MPAA rating
        - full release date
    Return information as a dictionary
    
    """
    
    base_url = 'https://www.boxofficemojo.com'
    
    #Create full url to scrape
    url = base_url + link
    
    #Request HTML and parse
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    
    headers = ['title', 'domestic_opening_gross', 'budget', 'domestic_gross', 'international_gross', 
               'worldwide_gross','runtime_minutes', 'rating', 'release_month', 'release_date','genres']
    
    #Get title
    title_string = soup.find('title').text
    title = title_string[:-18]

    #Get domestic gross
    raw_dom_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[0].text
    domestic_gross = money_to_int(raw_dom_gross)
    
    #dom_gross = money_to_int(dom_gross)
    
    #Get international_gross   
    try:
        raw_intl_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[1].text
        international_gross = money_to_int(raw_intl_gross)
    except:
        international_gross = 0
    
    #Get worldwide gross
    try:
        raw_world_gross = soup.find(class_='mojo-performance-summary-table').find_all('span',class_='money')[2].text
        worldwide_gross = money_to_int(raw_world_gross)
    except:
        worldwide_gross = 0
    
    #Get domestic opening gross
    try:
        raw_opening_gross = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[0].text
        domestic_opening_gross = money_to_int(raw_opening_gross)
    except:
        domestic_opening_gross = 0
    
    
    #Get budget
    try:
        raw_budget = soup.find(class_='mojo-summary-values').find_all('span', class_ = 'money')[1].text
        budget = money_to_int(raw_budget)
    except:
        budget = 0
    
    #Get distributor
    try:
        distributor = get_movie_value(soup, 'Distributor')[:18]
    except:
        distributor = ''
    
    #Get runtime
    raw_runtime = get_movie_value(soup, 'Running')
    runtime_minutes = runtime_to_minutes(raw_runtime)
    
    #Get rating
    rating = get_movie_value(soup, 'MPAA')
    
    #Get genres
    genres = get_movie_value(soup, 'Genres').split()

    
    #Get release month
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_month = to_date(raw_release_date).month
   
    #Get release date
    raw_release_date = get_movie_value(soup,'Release Date').split('\n')[0]
    release_date = to_date(raw_release_date)
    
    
    #Create movie dictionary and return
    movie_dict = dict(zip(headers, [title,
                                    domestic_opening_gross,
                                    budget,
                                    domestic_gross,
                                    international_gross,
                                    worldwide_gross,
                                    runtime_minutes,
                                    rating,
                                    release_month,
                                    release_date,
                                    genres]))

    return movie_dict

In [39]:
lifetime_movie_data = []
for link in lifetime_movies.link_stub:
    lifetime_movie_data.append(get_movie_dict(link))

In [40]:
lifetime_movie_data

[{'title': 'Star Wars: Episode VII - The Force Awakens',
  'domestic_opening_gross': 247966675,
  'budget': 245000000,
  'domestic_gross': 936662225,
  'international_gross': 1131791908,
  'worldwide_gross': 2068454133,
  'runtime_minutes': 138,
  'rating': 'PG-13',
  'release_month': 12,
  'release_date': datetime.datetime(2015, 12, 16, 0, 0),
  'genres': ['Action', 'Adventure', 'Sci-Fi']},
 {'title': 'Avengers: Endgame',
  'domestic_opening_gross': 357115007,
  'budget': 356000000,
  'domestic_gross': 858373000,
  'international_gross': 1939427564,
  'worldwide_gross': 2797800564,
  'runtime_minutes': 181,
  'rating': 'PG-13',
  'release_month': 4,
  'release_date': datetime.datetime(2019, 4, 24, 0, 0),
  'genres': ['Action', 'Adventure', 'Drama', 'Sci-Fi']},
 {'title': 'Avatar',
  'domestic_opening_gross': 77025481,
  'budget': 237000000,
  'domestic_gross': 760507625,
  'international_gross': 2029931467,
  'worldwide_gross': 2790439092,
  'runtime_minutes': 162,
  'rating': 'PG-13'

In [41]:
lifetime_movies_info = pd.DataFrame(lifetime_movie_data)
lifetime_movies_info.set_index('title', inplace = True)

In [111]:
# change number according to page
#lifetime_movies_info_5 = lifetime_movies_info
#lifetime_movies_5 = lifetime_movies
#lifetime_movies_5 = lifetime_movies_5.merge(lifetime_movies_info_5, left_index=True, right_index = True)
#lifetime_movies_5.head()

,link_stub,rank,title,domestic_gross_x,release_year,domestic_opening_gross,budget,domestic_gross_y,international_gross,worldwide_gross,runtime_minutes,rating,release_month,release_date,genres
Maid in Manhattan,/title/tt0252076/?ref_=bo_cso_table_1,801,Maid in Manhattan,"$94,011,225",2002,18711407,55000000,94011225,60895468,154906693,105.0,PG-13,12,2002-12-13,"[Comedy, Drama, Romance]"
He's Just Not That Into You,/title/tt1001508/?ref_=bo_cso_table_2,802,He's Just Not That Into You,"$93,953,653",2009,27785487,0,93953653,84912505,178866158,129.0,PG-13,2,2009-02-05,"[Comedy, Drama, Romance]"
Master and Commander: The Far Side of the World,/title/tt0311113/?ref_=bo_cso_table_3,803,Master and Commander: The Far Side of the World,"$93,927,920",2003,25105990,150000000,93927920,117694615,211622535,138.0,PG-13,11,2003-11-14,"[Action, Adventure, Drama, History, War]"
Flight,/title/tt1907668/?ref_=bo_cso_table_4,804,Flight,"$93,772,375",2012,24900566,31000000,93772375,68000000,161772375,138.0,R,11,2012-11-02,"[Drama, Thriller]"
The Fighter,/title/tt0964517/?ref_=bo_cso_table_5,805,The Fighter,"$93,617,009",2010,300010,25000000,93617009,35573860,129190869,116.0,R,11,2009-11-20,"[Biography, Drama, Sport]"


In [43]:
movie_data = pd.concat([lifetime_movies, lifetime_movies_info], axis=1)

In [44]:
movie_data


,link_stub,rank,title,domestic_gross,release_year,domestic_opening_gross,budget,domestic_gross,international_gross,worldwide_gross,runtime_minutes,rating,release_month,release_date,genres
title,,,,,,,,,,,,,,,
Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015,247966675,245000000,936662225,1131791908,2068454133,138.0,PG-13,12,2015-12-16,"[Action, Adventure, Sci-Fi]"
Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019,357115007,356000000,858373000,1939427564,2797800564,181.0,PG-13,4,2019-04-24,"[Action, Adventure, Drama, Sci-Fi]"
Avatar,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009,77025481,237000000,760507625,2029931467,2790439092,162.0,PG-13,12,2009-12-16,"[Action, Adventure, Fantasy, Sci-Fi]"
Black Panther,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018,202003951,0,700426566,647171407,1347597973,134.0,PG-13,2,2018-02-13,"[Action, Adventure, Sci-Fi]"
Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018,257698183,0,678815482,1369544272,2048359754,149.0,PG-13,4,2018-04-25,"[Action, Adventure, Sci-Fi]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Fast Five,/title/tt1596343/?ref_=bo_cso_table_196,196,Fast Five,"$209,837,675",2011,86198765,125000000,209837675,416300000,626137675,130.0,PG-13,4,2011-04-20,"[Action, Adventure, Crime, Thriller]"
Mission: Impossible - Ghost Protocol,/title/tt1229238/?ref_=bo_cso_table_197,197,Mission: Impossible - Ghost Protocol,"$209,397,903",2011,12785204,145000000,209397903,485315477,694713380,132.0,PG-13,12,2011-12-14,"[Action, Adventure, Thriller]"
Wedding Crashers,/title/tt0396269/?ref_=bo_cso_table_198,198,Wedding Crashers,"$209,255,921",2005,32200000,40000000,209255921,79211724,288467645,119.0,R,7,2005-07-14,"[Comedy, Romance]"


In [44]:
#lifetime_movies = lifetime_movies.merge(lifetime_movies_info, left_index=True, right_index=True)

#lifetime_movies.head()

,link_stub,rank,title,domestic_gross_x,release_year,domestic_opening_gross_x,budget_x,domestic_gross_y,international_gross_x,worldwide_gross_x,...,domestic_opening_gross_y,budget_y,domestic_gross,international_gross_y,worldwide_gross_y,runtime_minutes_y,rating_y,release_month_y,release_date_y,genres_y
Star Wars: Episode VII - The Force Awakens,/title/tt2488496/?ref_=bo_cso_table_1,1,Star Wars: Episode VII - The Force Awakens,"$936,662,225",2015,247966675,245000000,936662225,1131791908,2068454133,...,247966675,245000000,936662225,1131791908,2068454133,138.0,PG-13,12,2015-12-16,"[Action, Adventure, Sci-Fi]"
Avengers: Endgame,/title/tt4154796/?ref_=bo_cso_table_2,2,Avengers: Endgame,"$858,373,000",2019,357115007,356000000,858373000,1939427564,2797800564,...,357115007,356000000,858373000,1939427564,2797800564,181.0,PG-13,4,2019-04-24,"[Action, Adventure, Drama, Sci-Fi]"
Avatar,/title/tt0499549/?ref_=bo_cso_table_3,3,Avatar,"$760,507,625",2009,77025481,237000000,760507625,2029931467,2790439092,...,77025481,237000000,760507625,2029931467,2790439092,162.0,PG-13,12,2009-12-16,"[Action, Adventure, Fantasy, Sci-Fi]"
Black Panther,/title/tt1825683/?ref_=bo_cso_table_4,4,Black Panther,"$700,426,566",2018,202003951,0,700426566,647171407,1347597973,...,202003951,0,700426566,647171407,1347597973,134.0,PG-13,2,2018-02-13,"[Action, Adventure, Sci-Fi]"
Avengers: Infinity War,/title/tt4154756/?ref_=bo_cso_table_5,5,Avengers: Infinity War,"$678,815,482",2018,257698183,0,678815482,1369544272,2048359754,...,257698183,0,678815482,1369544272,2048359754,149.0,PG-13,4,2018-04-25,"[Action, Adventure, Sci-Fi]"


In [112]:
#change number page
lifetime_movies_5.to_csv('lifetime_movies_5.csv')

In [120]:
df1 = pd.read_csv('lifetime_movies.csv')
df2 = pd.read_csv('lifetime_movies_2.csv')
df3 = pd.read_csv('lifetime_movies_3.csv')
df4 = pd.read_csv('lifetime_movies_4.csv')
df5 = pd.read_csv('lifetime_movies_5.csv')
frames = [df1, df2, df3, df4, df5]
movie_list = pd.concat(frames)
movie_list.to_csv('movie_list.csv')
df = pd.read_csv('movie_list.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996 entries, 0 to 995
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              996 non-null    int64  
 1   Unnamed: 0.1            996 non-null    object 
 2   link_stub               996 non-null    object 
 3   rank                    996 non-null    object 
 4   title                   996 non-null    object 
 5   domestic_gross_x        996 non-null    object 
 6   release_year            996 non-null    int64  
 7   domestic_opening_gross  996 non-null    int64  
 8   budget                  996 non-null    int64  
 9   domestic_gross_y        996 non-null    int64  
 10  international_gross     996 non-null    int64  
 11  worldwide_gross         996 non-null    int64  
 12  runtime_minutes         975 non-null    float64
 13  rating                  867 non-null    object 
 14  release_month           996 non-null    in

### IMDB Top 100 Actors 2018

In [ ]:
url = 'https://www.imdb.com/list/ls023242359/'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [ ]:
soup.find_all('span', class_='lister-item-index')[0].text

In [ ]:
soup.find('div', class_='lister-item-content').find_all('a')

In [ ]:
soup.find('div', class_='lister-item-content').find('a').text.split('\n')[0]

In [ ]:
top_list = soup.find('div', class_='lister-list').find_all('a')
top_list

In [ ]:
soup.find('div', class_='lister-list').find_all('a')[1].text.split('\n')[0]

In [ ]:

top_actor = []
for top in top_list:
    top = top.text.split('\n')[0]
    top_actor += [top]
  
top_actor

In [ ]:
soup.find('div', class_='lister-list').find_all('a').get('href', ref_='nmls_hd')

In [ ]:
url='https://www.the-numbers.com/movie/budgets/all'
response = requests.get(url)
page = response.text
soup = BeautifulSoup(page, 'lxml')

In [ ]:
#soup.find_all('td', class_="data")

In [ ]:
# Rank:
rank = soup.find_all('td', class_="data")[0].text
rank

In [ ]:
# Budget
prod_budget = soup.find_all('td', class_="data")[1].text
prod_budget

In [ ]:
soup.find('table').find_all('td','href'=='/movie/')


In [ ]:
soup.find('table').find_all('a')[1]

In [ ]:
soup.find('table').find_all('a')[1].findNext()

In [ ]:
soup.find('table').find_all('href'=='/movie/*')